In [ ]:
%pip install -r requirements.txt

In [ ]:
import cv2
from PIL import Image
import numpy as np
import time
import torch
import matplotlib.pyplot as plt
import pandas as pd
import os
import imagehash
from binary import preprocessing
from model import VGG16
from predict import predict
from tensorflow.keras.models import load_model 

In [ ]:
device = torch.device('cuda')
vgg = VGG16(29)
vgg.load_state_dict(torch.load('models/best.pth', map_location=device))

In [ ]:
digit=load_model('models/digit.h5')

seg=torch.hub.load('ultralytics/yolov5', 'custom', path='models/amount.pt', force_reload=True) # nécessite la connection

In [ ]:


model=torch.hub.load('ultralytics/yolov5', 'custom', path='models/best.pt', force_reload=True)

In [ ]:
def detect(path,model):
    start=time.time()
    img=cv2.imread(path)
    d=img.copy()
    result=model(d,size=640)
    result.show()
    print(time.time()-start)

In [ ]:
def inference(img):
    result=model(img.copy(),size=640)
    df=result.pandas().xyxy[0]
    df=df.sort_values(by=['name','xmin'],ignore_index=True)
    return df

In [ ]:
def get_bbox(img,df,name):
    Box=[]
    if name=='bar':
        bbox=df.loc[df['name']==name][['xmin','ymin','xmax','ymax']].values.astype(int)
        
    elif (name=='word1') or (name=='word2'):
        bbox=df.loc[(df['name']==name) & (df['confidence']>=0.40)][['xmin','ymin','xmax','ymax']].values.astype(int)
        
    else:
        bbox=df.loc[(df['name']==name) & (df['confidence']>=0.45)][['xmin','ymin','xmax','ymax']].values.astype(int)
    for i in range(len(bbox)):
        x1,y1,x2,y2=bbox[i]
        Box.append(img.copy()[y1:y2,x1:x2])
    return  Box

In [ ]:
class detector:
    def __init__(self,path_image,vgg):
        self.img=cv2.imread(path_image)
        self.df=inference(self.img)
        self.sign=get_bbox(self.img,self.df,'sign')
        self.amount=get_bbox(self.img,self.df,'amount')
        self.word=get_bbox(self.img,self.df,'word1')
        w=get_bbox(self.img,self.df,'word2')
        self.word.extend(w)
        self.date=get_bbox(self.img,self.df,'date')
        self.place=get_bbox(self.img,self.df,'place')
        self.name=get_bbox(self.img,self.df,'name')
        self.plt_img=Image.fromarray(self.img)
        self.word=[preprocessing(img.copy()) for img in self.word]
        self.montant=[predict(img,vgg) for img in self.word ]
    def detect_bar(self):
        if len(self.df[self.df['name']=='bar'])==0:
            return False
        else:
            return True
    
    def verif_sign(self,imgs_database):
        if len(self.sign)==0:
            return 'Pas de signature trouve'
        else:
            score=np.zeros((len(self.sign),len(imgs_database)))
            for i in range(len(self.sign)):
                for j in range(len(imgs_database)):
                    img1=Image.fromarray(self.sign[i])
                    hash1 = imagehash.average_hash(img1)
                    hash2=imagehash.average_hash(Image.open(imgs_database[j]))
                    diff = abs(hash1 - hash2) # diff limite est 16 bits experimentalement
                    prob=1-(float(diff)/32)
                    score[i][j]=prob*100
            
            score=score.max(axis=1).flatten()
            score[score<0]=0
            
            return list(score)
    def montant_lettre(self):
        
        return ' '.join(self.montant)
    
    def montant_chiffre(self):
        a=self.amount[0]
        a=preprocessing(a.copy())
        result=seg(a.copy(),size=640)
        df=result.pandas().xyxy[0]
        df=df.sort_values(by=['name','xmin'],ignore_index=True)
        chars=get_bbox(a,df,'A')
        dic=dict(enumerate(list("0123456789A")))
        number=[]
        for image in chars:
            img=cv2.resize(image,(28,28))
            image=img.reshape(1,28,28,1)
            pred=dic[digit.predict(image).argmax()]
            if pred=='A':
                pred=''
            number.append(pred)
        number="".join(number)
        mont=int(number)
        return mont

In [ ]:
liste=[]
path='./ecobank/'
t=os.listdir(path)
for i in t:
    liste.append(os.path.join(path,i))

In [ ]:
a=liste[6]
detect(a,model) # Indique un apercu de comment l'algorithme récupère les données

In [ ]:
# Quelques cheques testés :146-147-148-150-184-188-189-190-191-192-195-203-205-209-210-213-215-216

In [ ]:
d=detector(a,vgg)
d.plt_img

In [ ]:
d.montant_lettre()

In [ ]:
d.detect_bar()

In [ ]:
d.verif_sign(['c.png'])

In [ ]:
d.montant_chiffre()

In [1]:
%pip freeze

absl-py @ file:///home/conda/feedstock_root/build_artifacts/absl-py_1658854572623/work
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-client @ file:///C:/ci/anaconda-client_1635342725944/work
anaconda-navigator==2.1.4
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1626085644852/work
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1660053721269/work/dist
appdirs==1.4.4
argh==0.26.2
argon2-cffi @ file:///C:/ci/argon2-cffi_1613037869401/work
arrow @ file:///C:/ci/arrow_1617738834352/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astor @ file:///home/conda/feedstock_root/build_artifacts/astor_1593610464257/work
astroid @ file:///C:/ci/astroid_1628063282661/work
astropy @ file:///C:/ci/astropy_1629829318700/work
asttokens==2.0.5
astunparse @ file:///home/conda/feedstock_root/build_artifacts/astunparse_1610696312422/work
async-generator @ file:///home/ktietz/src/ci/async_generator_1611927993394/work
ato